In [1]:
from typing import List

import torch
from torch import Tensor, nn

import pandas as pd

import sys
sys.path.insert(1, '/work/nlp-project')
from scripts.read_write_data import load_data

In [2]:
SAVE_PATH = "/work/nlp-project/models/Baseline.pt"
UNLABELED_PATH = "/work/nlp-project/data/processed/train_splits/unlabeled.conll"

In [3]:
docs, labels, _, _ = load_data(UNLABELED_PATH)

In [4]:
def pad_inputs(collection: List[List[int]], pad_token):
    to_series = [pd.Series(el) for el in collection]
    enc_matrix = (pd.concat(to_series, axis=1)
                    .reindex(range(100))
                    .fillna(pad_token)
                    .T)
    return enc_matrix.values.tolist()

In [5]:
pad_docs = pad_inputs(docs, pad_token='<PAD>')

In [6]:
model = torch.load(SAVE_PATH)

In [7]:
import numpy as np

outputs = []
intervals = np.arange(0, len(pad_docs), 32)
for start in intervals:
    end = start + 32
    batch_docs = pad_docs[start:end]
    preds = model.forward(batch_docs)
    outputs.extend(torch.argmax(preds, dim=2).tolist())

In [8]:
no_pad_output = []
for output, doc in zip(outputs, docs):
    trunc_otp = output[:len(doc)]
    no_pad_output.append(trunc_otp)

In [9]:
PSEUDO_OUTPUT_PATH = "/work/nlp-project/data/pseudo/pseudo.conll"
with open(PSEUDO_OUTPUT_PATH, encoding='utf-8', mode='w') as f:
    for doc, labs in zip(docs, no_pad_output):
        for word, lab in zip(doc, labs):
            f.write(word+'\t'+str(lab)+'\n')
        f.write('\n')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b2f14aee-af04-4db5-af55-57a3a58b9f40' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>